# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [3]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [4]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution
lst = ['play', 'ground']
print 'ground' in lst

m = matrix(QQ, [[1,2],[4,2],[11,3]])
print 'm = ',m
x = []
for v in m:
    x.append(v)
    
x = matrix(QQ, x)
print 'x = ',x
 
pr = Permutations(20).random_element()
print pr, pr[4], pr[7]


v = vector(QQ, [2,3,4])
print 'len = ',len(v)

min(5, 44)

 solution = [None, False, False, True]
True
m =  [ 1  2]
[ 4  2]
[11  3]
x =  [ 1  2]
[ 4  2]
[11  3]
[4, 9, 15, 10, 19, 16, 6, 5, 12, 2, 1, 17, 11, 7, 3, 8, 20, 13, 14, 18] 19 5
len =  3


5

In [5]:
# TODO: check correctness etc
def get_at(tup, a, l):
    """Given 2*l elements flattened into a tuple, pick 1 out of each pair according to array of bits."""
    assert len(tup) == 2*l
    assert len(a) == l
    for bit in a:
        assert bit in (0, 1)

    # tup structure is: $(t_{0,0}, t_{0,1}, t_{1,0}, t_{1,1}, t_{2,0}, t_{2,1})$
    return tuple(tup[2*i + a[i]] for i in range(l))

assert get_at((3, 13, 85, 95), (0, 1), 2) == (3, 95)

In [10]:
 
def tuples_fixed_proj(l, proj_ind, proj_val):
#    h = randint(1,4000)
#    if (h == 5):
#        print '==================== DEBUGGING ========== a,g = ', proj_ind, proj_val
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
#        if (h == 5):
#            print 'modified x to ', x
        tuples.append(tuple(x[:]))
    return tuples     

# Hack from correctness and server security (!) 
# strength = 0 is consistent with standard LP. strength = 1 attempts to better sort things out   

def add_no_id_a(solver, p0, q1, strength = 0):
    
    if strength == -1:
        return
    
    for i in range(3):                              
        for a in itertools.product(range(2), repeat = l):
            for j in range(3):
                if (j > i):
                    # this is relaxed
                    if strength == 0:
                        solver.add_constraint(sum(q1[(i, a, zeros, b)] for b in range(2)) + sum(q1[(j, a, zeros, b)] 
                                                 for b in range(2)) <= 1)    
                    elif strength == 1:
                        for b1 in range(2):
                            for b2 in range(2):
                                solver.add_constraint(p0[(i, a, zeros, b1)] + p0[(i, a, zeros, b2)] <= 1)
                                
# TODO: change the name to something more informative.
# Hack #2 from correctness: For every i,a,b with p^i_a > 0, there exists g so that (i,a,g,b) > 0.
# This requires some trial and error. Otherwise, one xi at least remains without TO-input options
# This will eventually not be part of the client constraints. Currently used to `direct' the soutions.
    
def add_a_balance(solver, p0, q1, strength = 0):
    if strength == -1:
        return
    
    if strength == 1:    
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - 1)
                solver.add_constraint(sum(p0[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) +
                                      sum(-p0[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l)) 
                                      >= 1 - pow(2, l))

    # Much weaker, but no integers..
    elif strength == 0:
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(q1[(i, a, g, 1)] for g in itertools.product(range(2), repeat = l)) 
                                      <= pow(2, l) - my_eps)
                solver.add_constraint(sum(q1[(i, a, g, 0)] for g in itertools.product(range(2), repeat = l))
                                      <= pow(2, l) - my_eps)


# Another hack from correctness. For all b, there exists at least one value of t, so 
# that every a reading this g will output b. Thus, the resulting sum of probabilities for
# that g is 1. There may be additional g's (the g results from some V assigned to server for each output
# value).                
def add_b_balance(solver, q1, strength = 0):
    if strength == -1:
        return 
    
    if strength == 0:
        for i in range(3):                              
            for b in range(2):
                # this is a relaxation - allows for splitting among several g's
                solver.add_constraint(sum(q1[(i, a, g, b)] for a in itertools.product(range(2), repeat = l) 
                                      for g in itertools.product(range(2), repeat = l)) >= 1)                


def get_rand_tup(len):
    return tuple([sage.misc.prandom.choice(range(2)) for i in range(len)])
                
def test_bounded_constr(l, client_bound):
    solver = MixedIntegerLinearProgram(solver='ppl')
    
    # Type 0
    p0 = solver.new_variable(integer = True, nonnegative=True)
    # Type 2
    q1 = solver.new_variable(integer = False, nonnegative=True)


    solver.add_constraint(sum(q1[(a, zeros, b)] for a in itertools.product(range(2), repeat = l)
                              for b in range(2)) == 3)

    # client's basic constraints                    
    for a in itertools.product(range(2), repeat = l):
        for g in itertools.product(range(2), repeat = l):
            for b in range(2):               
                solver.add_constraint(q1[(a,g,b)] <= 1)
                solver.add_constraint(p0[(a,g,b)] <= 1)
                solver.add_constraint(p0[(a,g,b)] - q1[(i,a,g,b)] >= 0)
                solver.add_constraint(p0[(a,g,b)] - q1[(i,a,g,b)] <= 1 - my_eps)

    # Look for a support as small as possible: to remove later.                
    solver.add_constraint(sum(p0[(i, a, g, b)] for i in range(3)
                          for a in itertools.product(range(2), repeat = l) 
                          for g in itertools.product(range(2), repeat = l)    
                          for b in range(2)) <= client_bound)               

    # Enforce the existence of p^i_a's 
    for a in itertools.product(range(2), repeat = l):
        for g in itertools.product(range(2), repeat = l):       
            if (g != zeros):
                solver.add_constraint(q1[(a,zeros,0)] + q1[(a,zeros,1)] 
                                      - q1[(a,g,0)] - q1[(a,g,1)] == 0)     

    # correctness constraints
    for t in itertools.product(range(2), repeat = 2*l):
        solver.add_constraint(sum(q1[(a, get_at(t,a,l) ,1)] 
                                  for a in itertools.product(range(2), repeat = l)) == 1)

    solver.solve()  
    
    for a in itertools.product(range(2), repeat = l):
        for g in itertools.product(range(2), repeat = l):
            for b in range(2):
                if solver.get_values(q1[(a, g, b)]) > 0:
                    print (a, g, b),': ',solver.get_values(q1[(a, g, b)]), '...', solver.get_values(p0[(i, a, g, b)])

                    
def safe_insert(val_l, ind_l, (i,ind), v):
    to_search = []
    if i >= 0:
        to_search = [i]
    else:
        to_search = [j for j in range(3)]
    
    for j in to_search:
        if (j,ind) in ind_l:
            val_l[ind_l.index((j,ind))] = v


def find_set_server_vars(l, p0):
    try:
        print 'How much is server variable space limited by this solution & correctness?'
        p1 = []
        for i in range(3):                              
            for t in itertools.product(range(2), repeat = 2*l):
                p1.append((i,t))

        for (y,(a,g,b)) in p0:
            for j in range(3):
                if ((b == 0) and (i == j)) or ((b == 1) and (i != j)):
                    all = tuples_fixed_proj(l, a, g)
                    for t in all:
                        try:
                            p1.remove((j, t))
                        except:
                            pass

        print 'number of remaining server values =',len(p1)
        if (len(p1) > 0):
            print 'server variables are ',p1
    except: 'Uncaught exception!'    


def permute_rows(m, b):
    
    nrows = m.nrows()
    perm = Permutations(nrows).random_element()
    pm = [row for row in m]
    pb = [v for v in b]
    
    new_m = matrix(QQ, [pm[perm[i] - 1] for i in range(nrows)])
    
    
    new_b = vector(QQ, [pb[perm[i] - 1] for i in range(nrows)])
    
    return (new_m, new_b)
    
    
    
def get_full_rank_sub(m, b):
   
    nrows = m.nrows()
    new_mat = []
    new_b = []
    
 #   print 'into get_full_rank_sub',m.nrows(), m.ncols()
    
    for (i,row) in enumerate(m):
        so_far = (matrix(QQ, new_mat)).transpose()
        try:
            so_far\row
        except Exception as e:
   #         print 'at full rank sub Exception = ',e
            new_mat.append(row)
            new_b.append(b[i])
        
    new_mat = matrix(QQ, new_mat)
    new_b = vector(new_b)
    
    return (new_mat, new_b)


def print_matrix(m):

    per_row = 30
    for row in m:
        n = ceil(QQ(m.nrows())/per_row)
        for i in range(n):
            ran = min(per_row, m.nrows() - i * per_row)
            print [row[i * per_row + j] for j in range(ran)]
        print '\n'    
          

def non_zeros(m):            
    return sum(sum(1 for j in row if abs(j) != 0) for row in m)        
        
        
def get_inv_sub_matrix(m, b):
    
    (new_mat, new_b) =  get_full_rank_sub(m, b)
    
    ncols = new_mat.ncols()
    print 'new_mat rank /  ',new_mat.rank(),'(',new_mat.nrows(),',',new_mat.ncols(),')'
    
    new_mat_tr = new_mat.transpose()
    b_dum = vector(QQ, [0 for i in range(ncols)]) 
    
    
    (full_sub, b_dum) = permute_rows(new_mat_tr, b_dum)
    
    print 'Before get_full_rank'
    
    (full_sub, b_dum) = get_full_rank_sub(full_sub, b_dum)
    
    inv_sub = full_sub.inverse()
    
    print 'x = A^{-1}*b',
 #   print full_sub
    
    det_A = full_sub.determinant()
    print '|A| = ', det_A
    # Finding independent columns
    print 'Sparsity parameters',non_zeros(full_sub),' => ',non_zeros(inv_sub)
    
    print 'x = ',inv_sub * new_b
    
    if (full_sub.nrows() < 500 and abs(det_A) > 1):
        print print_matrix(inv_sub)
    
# only captures a small subset of constraints                
def test_linear_solutions(l, strength = 0):
    
        b = []
        constraints = []
        
        print 'generating constraints ...'
        
        # pick 3 t's for x1, x2, x3 at random
        ts = [get_rand_tup(2*l) for i in range(3)]
        a_set = []
        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l):
                for z in range(2):
                    i = sage.misc.prandom.choice(range(3))
                    a_set.append((i,(a,g,z)))
        
        
        if strength > 0:
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):
                    for j in range(3):
                        for i in range(3):
                            cur_tup = (a, get_at(ts[j],a,l), 1 - (i == j))
                            if (i, cur_tup) in a_set:
                                a_set.remove((i, cur_tup))

         
        a_set_len = len(a_set)
        # needs strengthening into 3 equations
        for i in range(3):
            cur_row = [0 for j in range(a_set_len)]
            for a in itertools.product(range(2), repeat = l):
                for z in range(2):
                    safe_insert(cur_row, a_set, (i, (a, zeros, z)), 1)
                
            constraints.append(cur_row)
            b.append(1)


        for a in itertools.product(range(2), repeat = l):
            for g in itertools.product(range(2), repeat = l): 
                if g != zeros:
                    cur_row = [0 for i in range(a_set_len)]
                    safe_insert(cur_row, a_set, (-1, (a, zeros, 0)), 1)
                    safe_insert(cur_row, a_set, (-1, (a, zeros, 1)), 1)
                    safe_insert(cur_row, a_set, (-1, (a, g, 0)), -1)
                    safe_insert(cur_row, a_set, (-1, (a, g, 1)), -1)
                    b.append(0)
                    constraints.append(cur_row)

                    
        for t in itertools.product(range(2), repeat = 2*l):
            cur_row = [0 for i in range(a_set_len)]
            b.append(1)
            for a in itertools.product(range(2), repeat = l):
                safe_insert(cur_row, a_set, (-1, (a, get_at(t,a,l), 1)), 1)
            constraints.append(cur_row)
            
        
        m_constr = matrix(QQ,constraints)
        right_side = vector(QQ,b)
        print 'Created linear system of rank',m_constr.rank(),'/',pow(2, 2*l + 1),' vs ',a_set_len
        try:
            v = m_constr\right_side
#            ker = m_constr.right_kernel()        
            print 'solution = ',v
            nz = sum(1 for x in v if abs(x) > 0)
            print 'non-zeros ', nz
#           print 'Kernel = ', ker
            
            for (i,val) in enumerate(a_set):
                if v[i] == 0:
                    a_set.remove(val)
            find_set_server_vars(l, a_set)
            get_inv_sub_matrix(m_constr, right_side)
            
            print '==================================================V'
        except Exception as e:
            print 'Exception = ',e
          #      print 'There is no solution!'
            print '==================================================X'
            return (10000, a_set_len)
        
        print 'Returning ',(nz, a_set_len)
        return (nz, a_set_len)        


# global variables

my_eps = QQ(1/10000)

In [ ]:
# main     

import time    
min_nz = 10000
min_out_of = -1                    
for l in range(4, 5):
    for step in range(4):
        t0 = time.time()
        print 'finding a solution for l=',l
        zeros = tuple((0 for i in range(l)))

        try:
            (nz, out_of) = test_linear_solutions(l, 0)
            if nz < min_nz:   
                min_nz = nz
                min_out_of = out_of    
        finally:
            print 'time =', time.time() - t0,'\n'

 
print 'Minimum is ', (min_nz, min_out_of)

finding a solution for l= 4
generating constraints ...
Created linear system of rank 324 / 512  vs  512
solution =  (5/2, 1/2, 3, 0, 3/2, 3/2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/2, -1/2, 0, 0, 1/2, -1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0]
[0, -1, -1, 0, 0, -1/2, 0, 0, 0, 0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 1, 0, 0, 0, 1, 0, 0, -1, -1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1/2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1/2, 1, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, -1/2, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1/2, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1/2, 0, 1, -1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -1, 1, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, 1, 1/2, 0, 0, -1/2, 0, 1, -1/2]
[0, 1/2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0, 0, 0, -1, 0]
[-1, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1/2, 1, 1, -1, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

[0, 0, 0, 1, -1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, 1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0

Created linear system of rank 324 / 512  vs  512
solution =  (1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0

[0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0,

Created linear system of rank 324 / 512  vs  512
solution =  (1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

##### 